# Установка библиотек

In [ ]:
!pip install easyocr

In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.1 MB/s 


## Загрузим необоходимые библиотеки

In [ ]:
from imutils.video import VideoStream
from imutils.video import FPS
import numpy as np
import argparse
import imutils
from sklearn.metrics import classification_report, recall_score, accuracy_score
import time
import cv2
import matplotlib.pyplot as plt 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import os
from tqdm.notebook import tqdm
import logging
import pickle
import datetime
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
import subprocess as sp
from torch.utils.data import Dataset, DataLoader
import copy
from functools import lru_cache
from sklearn.model_selection import train_test_split
import catboost as cb
from PIL import ImageFilter, ImageChops

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
WORKING_DIR = '/content/drive/MyDrive/ai/video_surveillance/'
FRAME_DATA_FILE = os.path.join(WORKING_DIR, 'datasets/frame_data.pkl')
PEOPLE_DATA_FILE = os.path.join(WORKING_DIR, 'datasets/people_data.pkl')
# http://zulko.github.io/blog/2013/09/27/read-and-write-video-frames-in-python-using-ffmpeg/
FFMPEG_BIN = "ffmpeg"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
!ffmpeg -y -i {os.path.join(WORKING_DIR, 'datasets/train/train1.avi')} -filter:v "setpts=0.1*PTS" {os.path.join(WORKING_DIR, 'datasets/train/train1_fast.mp4')}
!ffmpeg -y -i {os.path.join(WORKING_DIR, 'datasets/train/train2.avi')} -filter:v "setpts=0.1*PTS" {os.path.join(WORKING_DIR, 'datasets/train/train2_fast.mp4')}
!ffmpeg -y -i {os.path.join(WORKING_DIR, 'datasets/test/test.avi')} -filter:v "setpts=0.1*PTS" {os.path.join(WORKING_DIR, 'datasets/test/test_fast.mp4')}

In [ ]:
!ffmpeg -y -i {os.path.join(WORKING_DIR, 'datasets/train/train1_fast.mp4')} -filter:v "crop=750:100:0:100" {os.path.join(WORKING_DIR, 'datasets/train/train1_fast_date.mp4')}
!ffmpeg -y -i {os.path.join(WORKING_DIR, 'datasets/train/train2_fast.mp4')} -filter:v "crop=570:70:0:50" {os.path.join(WORKING_DIR, 'datasets/train/train2_fast_date.mp4')}
!ffmpeg -y -i {os.path.join(WORKING_DIR, 'datasets/test/test_fast.mp4')} -filter:v "crop=570:70:0:50" {os.path.join(WORKING_DIR, 'datasets/train/test_fast_date.mp4')}

In [ ]:
!ffprobe -v error -select_streams v:0 -count_packets -show_entries stream=nb_read_packets -of csv=p=0 {os.path.join(WORKING_DIR, 'datasets/train/train1_fast.mp4')}
!ffprobe -v error -select_streams v:0 -count_packets -show_entries stream=nb_read_packets -of csv=p=0 {os.path.join(WORKING_DIR, 'datasets/train/train2_fast.mp4')}
!ffprobe -v error -select_streams v:0 -count_packets -show_entries stream=nb_read_packets -of csv=p=0 {os.path.join(WORKING_DIR, 'datasets/test/test_fast.mp4')}

79168
120211
74760


In [ ]:
!ffprobe -v error -select_streams v:0 -count_packets -show_entries stream=nb_read_packets -of csv=p=0 {os.path.join(WORKING_DIR, 'datasets/train/train1_fast_date.mp4')}
!ffprobe -v error -select_streams v:0 -count_packets -show_entries stream=nb_read_packets -of csv=p=0 {os.path.join(WORKING_DIR, 'datasets/train/train2_fast_date.mp4')}
!ffprobe -v error -select_streams v:0 -count_packets -show_entries stream=nb_read_packets -of csv=p=0 {os.path.join(WORKING_DIR, 'datasets/test/test_fast_date.mp4')}

79168
120211
74760


# Присвоение каждой секунде кадра соответствующее время

In [ ]:
datasets_videos_fast = {'train/train1_fast.mp4': [79168, [2560, 1920]], 
                        'train/train2_fast.mp4': [120211, [1920, 1080]], 
                        'test/test_fast.mp4': [74760, [1920, 1080]]
                        }
datasets_videos_fast_date = {'train/train1_fast_date.mp4': [79168, [750, 100], [[0, 0, 340, 100], [460, 0, 750, 100]]], 
                             'train/train2_fast_date.mp4': [120211, [570, 70], [[0, 0, 250, 70], [350, 0, 570, 70]]], 
                             'test/test_fast_date.mp4': [74760, [570, 70], [[0, 0, 250, 70], [350, 0, 570, 70]]]
                             }

In [ ]:
from easyocr import Reader
reader = Reader(['en'], gpu=True)

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

In [ ]:
frame_data = {}

for dataset, (frames_count, size_img, rects) in datasets_videos_fast_date.items():
    if dataset not in frame_data:
        frame_data[dataset] = {}
    command = [ FFMPEG_BIN,
            '-i', os.path.join(WORKING_DIR, f'datasets/{dataset}'),
            '-f', 'image2pipe',
            '-pix_fmt', 'rgb24',
            '-vcodec', 'rawvideo', '-']
    pipe = sp.Popen(command, stdout=sp.PIPE)
    for frame_i in tqdm(range(frames_count)):
        raw_frame = pipe.stdout.read(size_img[0] * size_img[1] * 3)
        #frame = np.fromstring(raw_frame, dtype='uint8')
        frame = np.frombuffer(raw_frame, dtype='uint8')
        if not any(frame):
            break
        frame = frame.reshape((size_img[1], size_img[0], 3))

        x1, y1, x2, y2 = rects[0]
        frame_date = frame[y1:y2, x1:x2]
        x1, y1, x2, y2 = rects[1]
        frame_time = frame[y1:y2, x1:x2]

        frame_date = cv2.cvtColor(frame_date, cv2.COLOR_RGB2BGR)
        frame_time = cv2.cvtColor(frame_time, cv2.COLOR_RGB2BGR)

        gray = cv2.cvtColor(frame_date, cv2.COLOR_BGR2GRAY)
        frame_date = cv2.GaussianBlur(gray, (7, 7), 0)
        gray = cv2.cvtColor(frame_time, cv2.COLOR_BGR2GRAY)
        frame_time = cv2.GaussianBlur(gray, (7, 7), 0)

        txt_date = reader.readtext(frame_date)
        txt_time = reader.readtext(frame_time)
        if len(txt_date) == 1 and len(txt_time) == 1:
            txt_date = txt_date[0][1].replace('Q', '0').replace('O', '0')
            txt_time = txt_time[0][1].replace('Q', '0').replace('O', '0')
        else:
            txt_date = ''
            txt_time = ''

        frame_data[dataset][frame_i] = [txt_date, txt_time]

        if frame_i % 1000 == 0:
            pickle.dump(frame_data, open(FRAME_DATA_FILE, 'wb'))
    pickle.dump(frame_data, open(FRAME_DATA_FILE, 'wb'))

# Постобработка frame_data

In [ ]:
frame_data = pickle.load(open(FRAME_DATA_FILE, 'rb'))

In [ ]:
for dates_data in frame_data.values():
    for frame_i, date_data in list(dates_data.items()):
        date, time = date_data
        date = ''.join([elem for elem in date if elem in '01234567890'])
        time = ''.join([elem for elem in time if elem in '01234567890'])
        if len(date) != 8 or len(time) != 6:
            del dates_data[frame_i]
            continue
        date = f'{date[4:].replace("2027", "2022")}-{date[:2]}-{date[2:4]}'
        time = f'{time[:2]}:{time[2:4]}:{time[4:]}'
        dates_data[frame_i] = ' '.join([date, time])

In [ ]:
frame_data['train/train1_fast.mp4'] = frame_data['train/train1_fast_date.mp4']
frame_data['train/train2_fast.mp4'] = frame_data['train/train2_fast_date.mp4']
frame_data['test/test_fast.mp4'] = frame_data['test/test_fast_date.mp4']

In [ ]:
del frame_data['train/train1_fast_date.mp4']
del frame_data['train/train2_fast_date.mp4']
del frame_data['test/test_fast_date.mp4']

In [ ]:
pickle.dump(frame_data, open(FRAME_DATA_FILE, 'wb'))

# Немного поздновато решил уменьшить качество train1 (надо было раньше)

In [ ]:
!ffmpeg -y -i {os.path.join(WORKING_DIR, 'datasets/train/train1_fast.mp4')} -vf scale=1920:1080 {os.path.join(WORKING_DIR, 'datasets/train/train1_resized_fast.mp4')}

In [ ]:
frame_data = pickle.load(open(FRAME_DATA_FILE, 'rb'))
frame_data['train/train1_resized_fast.mp4'] = frame_data['train/train1_fast.mp4']
del frame_data['train/train1_fast.mp4']
pickle.dump(frame_data, open(FRAME_DATA_FILE, 'wb'))

# Предсказание для картинок

In [ ]:
datasets_videos_fast = {'train/train1_resized_fast.mp4': [79168, [1920, 1080]], 
                        'train/train2_fast.mp4': [120211, [1920, 1080]], 
                        'test/test_fast.mp4': [74760, [1920, 1080]]
                        }

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path=os.path.join(WORKING_DIR, 'yolo_human_detect.pt'))
model.classes = [0]
model.conf = 0.3

In [ ]:
people_data = pickle.load(open(os.path.join(WORKING_DIR, 'datasets/people_data.pkl'), 'rb'))

for dataset, (frames_count, size_img) in datasets_videos_fast.items():
    if dataset not in people_data:
        people_data[dataset] = {}
    command = [FFMPEG_BIN,
            '-i', os.path.join(WORKING_DIR, f'datasets/{dataset}'),
            '-f', 'image2pipe',
            '-pix_fmt', 'rgb24',
            '-vcodec', 'rawvideo', '-']
    pipe = sp.Popen(command, stdout=sp.PIPE)
    for frame_i in tqdm(range(frames_count)):
        raw_frame = pipe.stdout.read(size_img[0] * size_img[1] * 3)
        frame = np.frombuffer(raw_frame, dtype='uint8')
        if frame_i in people_data[dataset]:
            continue
        #print('a')
        if not any(frame):
            break
        frame = frame.reshape((size_img[1], size_img[0], 3))

        preds = model(frame).xyxy[0]

        if preds.any():
            people_data[dataset][frame_i] = preds.tolist()

        if frame_i % 1000 == 0:
            pickle.dump(people_data, open(PEOPLE_DATA_FILE, 'wb'))
    pickle.dump(people_data, open(PEOPLE_DATA_FILE, 'wb'))

# Обработка csv датасета

In [ ]:
train_base_df = pd.read_csv(os.path.join(WORKING_DIR, 'datasets/train/train.csv'))
test_base_df = pd.read_csv(os.path.join(WORKING_DIR, 'datasets/test/test.csv'))

In [ ]:
# action 10 = движение при выключенном свете
train_base_df[train_base_df['action'] == 10]

,id,date,employee,action
563,1819,2022-05-25 01:11:34,0,10.0
572,1828,2022-05-25 01:55:12,0,10.0


In [ ]:
# сотрудника №9 впринципе нет, поэтому его стоит потом удалить
train_base_df[train_base_df['employee'] == 9]

,id,date,employee,action


In [ ]:
# action 0 = непонятное действие
train_base_df[train_base_df['action'] == 0]

,id,date,employee,action
1216,2472,2022-05-26 06:53:12,2,0.0
1217,2473,2022-05-26 06:53:29,2,0.0
1233,2489,2022-05-26 07:06:47,10,0.0
1234,2490,2022-05-26 07:06:53,10,0.0
1374,2630,2022-05-26 10:07:56,0,0.0
1411,2667,2022-05-26 10:40:57,0,0.0
1412,2668,2022-05-26 10:41:12,0,0.0
1419,2675,2022-05-26 10:42:13,0,0.0
1435,2691,2022-05-26 11:05:26,0,0.0
2118,3374,2027-05-27 00:18:52,0,0.0


In [ ]:
# что за 2027 год!?!?!?!?! Люди в Иркутске, мы что-то о вас не знаем?
train_base_df[train_base_df['date'].str.contains('2027-')]

,id,date,employee,action
2094,3350,2027-05-27 00:00:42,0,5.0
2095,3351,2027-05-27 00:01:14,0,15.0
2096,3352,2027-05-27 00:01:31,0,2.0
2097,3353,2027-05-27 00:01:49,0,12.0
2098,3354,2027-05-27 00:02:23,8,12.0
...,...,...,...,...
2215,3471,2027-05-27 07:33:05,3,16.0
2216,3472,2027-05-27 07:33:13,3,4.0
2217,3473,2027-05-27 07:33:14,6,5.0
2218,3474,2027-05-27 07:33:45,3,14.0


In [ ]:
test_base_df[test_base_df['date'].str.contains('2027-')]

,id,date


In [ ]:
indexes_2027 = train_base_df[train_base_df['date'].str.contains('2027-')].index
for i in indexes_2027:
    train_base_df.at[i, 'date'] = '2022' + train_base_df.at[i, 'date'][4:]

In [ ]:
train_base_df['employee'] = train_base_df['employee'].replace(10, 9).replace(11, 10)

# Инициализация кэшированной даты

In [ ]:
frame_data = pickle.load(open(FRAME_DATA_FILE, 'rb'))
people_data = pickle.load(open(PEOPLE_DATA_FILE, 'rb'))
head_data = pickle.load(open(HEAD_DATA_FILE, 'rb'))

In [ ]:
train_dates = {key: val for dataset in ['train/train1_resized_fast.mp4', 'train/train2.mp4'] for key, val in frame_data.items()}
train_peoples = {key: val for dataset in ['train/train1_resized_fast.mp4', 'train/train2.mp4'] for key, val in people_data.items()}

In [ ]:
test_dates = {key: val for dataset in ['test/test_fast.mp4'] for key, val in frame_data.items()}
test_peoples = {key: val for dataset in ['test/test_fast.mp4'] for key, val in people_data.items()}

# Детектинг людей вышли/зашли

In [ ]:
datasets = []
for i in range(len(train_base_df)):
    row = train_base_df.iloc[i]
    date = row['date']
    if date in frame_data['train/train1_resized_fast.mp4'].values():
        datasets.append('train/train1_resized_fast.mp4')
    elif date in frame_data['train/train2_fast.mp4'].values():
        datasets.append('train/train2_fast.mp4')
    else:
        datasets.append(None)
train_base_df['dataset'] = datasets
len(train_base_df), len(train_base_df[train_base_df['dataset'].notnull()])

(3519, 3002)

In [ ]:
max_n_people = 26
cb_train_action_df = pd.DataFrame({}, columns=[f'frame_{i}' for i in range(max_n_people)] + ['label'])
cb_train_door_df = pd.DataFrame({}, columns=['frame_i', 'door', 'x1', 'y1', 'x2', 'y2'])
pred_actions = []
for i in range(len(train_base_df)):
    row = train_base_df.iloc[i]
    dataset = row['dataset']
    date = row['date']
    action = row['action']
    employee = row['employee']
    if dataset is None or action in [0, 10]:
        continue

    from_frame_i = min([frame_i for frame_i, date_ in frame_data[dataset].items() if date == date_])
    if from_frame_i - 10 >= 0:
        from_frame_i -= 10
    else:
        from_frame_i = 0
    to_frame_i = max([frame_i for frame_i, date_ in frame_data[dataset].items() if date == date_]) + 5
    if to_frame_i + 5 <= max(frame_data[dataset]):
        to_frame_i += 5
    else:
        to_frame_i = max(frame_data[dataset])
    n_people = []
    for frame_i in range(from_frame_i, to_frame_i + 1):
        if frame_i not in people_data[dataset]:
            n_people.append(0)
            continue
        annotations = people_data[dataset][frame_i]
        n_people.append(len(annotations))
    n_people = n_people[:max_n_people]
    for j in range(len(n_people) - 2):
        if n_people[j] != n_people[j + 1] and n_people[j] == n_people[j + 2]:
            n_people[j + 1] = n_people[j]
    
    mid_i = len(n_people) // 2
    if action > 10:
        # список индексов, где появляется человек до центра
        to_middle = []
        for human_i in range(1, mid_i + 1):
            if n_people[human_i] > n_people[human_i - 1]:
                to_middle.append(human_i)
        if not any(to_middle):
            for human_i in range(mid_i + 1, len(n_people)):
                if n_people[human_i] > n_people[human_i - 1]:
                    to_middle.append(human_i)
            if not any(to_middle):
                continue
            human_i = min(to_middle)
        else:
            human_i = max(to_middle)
    else:
        # список индексов, где исчезает за дверью человек до центра
        to_middle = []
        for human_i in range(mid_i + 1, len(n_people)):
            if n_people[human_i] < n_people[human_i - 1]:
                to_middle.append(human_i)
        if not any(to_middle):
            for human_i in range(1, mid_i + 1):
                if n_people[human_i] < n_people[human_i - 1]:
                    to_middle.append(human_i)
            if not any(to_middle):
                continue
            human_i = max(to_middle)
        else:
            human_i = min(to_middle)
    
    cheking_n_people = n_people[human_i - 1:human_i + 1]
    if abs(cheking_n_people[0] - cheking_n_people[1]) != 1:
        continue
    if 0 in cheking_n_people and 1 in cheking_n_people:
        if action > 10:
            frame_i = from_frame_i + human_i
        else:
            frame_i = from_frame_i + human_i - 1
        annotation = people_data[dataset][frame_i][0]
    else:
        if action > 10:
            frame_i = from_frame_i + human_i
            annotations1 = people_data[dataset][frame_i - 1]
            annotations2 = people_data[dataset][frame_i]
        else:
            frame_i = from_frame_i + human_i - 1
            annotations1 = people_data[dataset][frame_i + 1]
            annotations2 = people_data[dataset][frame_i]
        center_annotation1 = [[annotation[0] + (annotation[2] - annotation[0]) // 2,
                               annotation[1] + (annotation[3] - annotation[1]) // 2]
                              for annotation in annotations1]
        center_annotation2 = [[annotation[0] + (annotation[2] - annotation[0]) // 2,
                               annotation[1] + (annotation[3] - annotation[1]) // 2]
                              for annotation in annotations2]
        for annotation1 in center_annotation1:
            len_to_center = []
            for annotation2 in center_annotation2:
                len_to_center.append(((annotation1[0] - annotation2[0]) ** 2 + (annotation1[1] - annotation2[1]) ** 2) ** 0.5)
            del_i = len_to_center.index(min(len_to_center))
            del annotations2[del_i]
            del center_annotation2[del_i]
        if len(center_annotation2) != 1:
            continue
        annotation = annotations2[0]

    action_frame_row = pd.Series([frame_i, action if action < 10 else action - 10, annotation[0], annotation[1], annotation[2], annotation[3]], index=cb_train_door_df.columns)
    cb_train_door_df = cb_train_door_df.append(action_frame_row, ignore_index=True)
    #print(from_frame_i, human_i)


    #print(action, n_people)
    n_people.extend([0] * (max_n_people - len(n_people)))
    train_row = pd.Series(n_people + [int(action > 10)], index=cb_train_action_df.columns)
    cb_train_action_df = cb_train_action_df.append(train_row, ignore_index=True)
    #if not all([elem == 1 for elem in n_people]):
    #    print(n_people)

In [ ]:
x_train_action, x_test_action, y_train_action, y_test_action = train_test_split(cb_train_action_df.drop(columns=['label']), cb_train_action_df['label'], test_size=0.2)
x_train_door, x_test_door, y_train_door, y_test_door = train_test_split(cb_train_door_df.drop(columns=['door', 'frame_i']), cb_train_door_df['door'], test_size=0.2)

In [ ]:
train_dataset_action = cb.Pool(x_train_action, y_train_action)
train_dataset_door = cb.Pool(x_train_door, y_train_door)

In [ ]:
eval_dataset_action = cb.Pool(x_test_action, y_test_action)
eval_dataset_door = cb.Pool(x_test_door, y_test_door)

In [ ]:
model_action = cb.CatBoostClassifier(loss_function='Logloss', eval_metric='Recall', iterations=1000)
model_door = cb.CatBoostClassifier(loss_function='MultiClass', eval_metric='Accuracy', iterations=1000)

In [ ]:
model_action.fit(train_dataset_action)

In [ ]:
model_door.fit(train_dataset_door)

In [ ]:
pred = model_action.predict(x_test_action)
(y_test_action == pred).mean()

0.9333333333333333

In [ ]:
pred = model_door.predict(x_test_door).flatten()
(y_test_door == pred).mean()

In [ ]:
model_door.save_model(os.path.join(WORKING_DIR, 'model_door.cb'))
model_action.save_model(os.path.join(WORKING_DIR, 'model_action.cb'))

# Предсказание дверей для тестовой выборки

In [ ]:
model_action = cb.CatBoostClassifier()
model_door = cb.CatBoostClassifier()

In [ ]:
model_door = model_door.load_model(os.path.join(WORKING_DIR, 'model_door.cb'))
model_action = model_action.load_model(os.path.join(WORKING_DIR, 'model_action.cb'))

In [ ]:
test_base_df['dataset'] = 'test/test_fast.mp4'

In [ ]:
max_n_people = 26
actions = []
for i in range(len(test_base_df)):
    row = test_base_df.iloc[i]
    dataset = row['dataset']
    date = row['date']
    actions.append(0)
    if date not in frame_data[dataset].values():
        continue
    from_frame_i = min([frame_i for frame_i, date_ in frame_data[dataset].items() if date == date_])
    if from_frame_i - 10 >= 0:
        from_frame_i -= 10
    else:
        from_frame_i = 0
    to_frame_i = max([frame_i for frame_i, date_ in frame_data[dataset].items() if date == date_]) + 5
    if to_frame_i + 5 <= max(frame_data[dataset]):
        to_frame_i += 5
    else:
        to_frame_i = max(frame_data[dataset])
    n_people = []
    for frame_i in range(from_frame_i, to_frame_i + 1):
        if frame_i not in people_data[dataset]:
            n_people.append(0)
            continue
        annotations = people_data[dataset][frame_i]
        n_people.append(len(annotations))
    n_people = n_people[:max_n_people]
    for j in range(len(n_people) - 2):
        if n_people[j] != n_people[j + 1] and n_people[j] == n_people[j + 2]:
            n_people[j + 1] = n_people[j]
    
    predictig_n_people = n_people.copy()
    predictig_n_people.extend([0] * (max_n_people - len(n_people)))
    action = model_action.predict(predictig_n_people) * 10
    
    mid_i = len(n_people) // 2
    if action == 10:
        # список индексов, где появляется человек до центра
        to_middle = []
        for human_i in range(1, mid_i + 1):
            if n_people[human_i] > n_people[human_i - 1]:
                to_middle.append(human_i)
        if not any(to_middle):
            for human_i in range(mid_i + 1, len(n_people)):
                if n_people[human_i] > n_people[human_i - 1]:
                    to_middle.append(human_i)
            if not any(to_middle):
                continue
            human_i = min(to_middle)
        else:
            human_i = max(to_middle)
    else:
        # список индексов, где исчезает за дверью человек до центра
        to_middle = []
        for human_i in range(mid_i + 1, len(n_people)):
            if n_people[human_i] < n_people[human_i - 1]:
                to_middle.append(human_i)
        if not any(to_middle):
            for human_i in range(1, mid_i + 1):
                if n_people[human_i] < n_people[human_i - 1]:
                    to_middle.append(human_i)
            if not any(to_middle):
                continue
            human_i = max(to_middle)
        else:
            human_i = min(to_middle)
    
    cheking_n_people = n_people[human_i - 1:human_i + 1]
    if abs(cheking_n_people[0] - cheking_n_people[1]) != 1:
        continue
    if 0 in cheking_n_people and 1 in cheking_n_people:
        if action == 10:
            frame_i = from_frame_i + human_i
        else:
            frame_i = from_frame_i + human_i - 1
        annotation = people_data[dataset][frame_i][0]
    else:
        if action == 10:
            frame_i = from_frame_i + human_i
            annotations1 = people_data[dataset][frame_i - 1]
            annotations2 = people_data[dataset][frame_i]
        else:
            frame_i = from_frame_i + human_i - 1
            annotations1 = people_data[dataset][frame_i + 1]
            annotations2 = people_data[dataset][frame_i]
        center_annotation1 = [[annotation[0] + (annotation[2] - annotation[0]) // 2,
                               annotation[1] + (annotation[3] - annotation[1]) // 2]
                              for annotation in annotations1]
        center_annotation2 = [[annotation[0] + (annotation[2] - annotation[0]) // 2,
                               annotation[1] + (annotation[3] - annotation[1]) // 2]
                              for annotation in annotations2]
        for annotation1 in center_annotation1:
            len_to_center = []
            for annotation2 in center_annotation2:
                len_to_center.append(((annotation1[0] - annotation2[0]) ** 2 + (annotation1[1] - annotation2[1]) ** 2) ** 0.5)
            del_i = len_to_center.index(min(len_to_center))
            del annotations2[del_i]
            del center_annotation2[del_i]
        if len(center_annotation2) != 1:
            continue
        annotation = annotations2[0]
    
    action += int(model_door.predict(annotation))
    actions[-1] = action

In [ ]:
test_base_df['action'] = actions
test_base_df['employee'] = 0

In [ ]:
for n in range(1, 10):
    value_counts = train_base_df[train_base_df['action'].isin([n, n + 10])]['employee'].value_counts()
    if not any(value_counts.index):
        continue
    max_employee = value_counts.index[0]
    test_base_df.loc[test_base_df['action'].isin([n, n + 10]), 'employee'] = max_employee

In [ ]:
test_base_df.drop(columns=['dataset', 'date']).to_csv('out.csv', index=False)